<a href="https://colab.research.google.com/github/RizaCode/RizaCode/blob/main/Project_DevLaunch_Ariza_Sulthan_Rafi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q streamlit
!pip install python-dotenv
!npm install localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.3 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
added 22 packages in 2s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸

In [ ]:
%%writefile bot.py

def customer_service(query: str) -> str:
    """Bot ekspor-impor untuk RJL."""
    query = query.lower()

    if "apa yang" in query or "halo" in query:
        return """Bot ini bisa membantu Anda dengan:\n
        1. 📦 Informasi tentang perusahaan kami\n
        2. 🚢 Jenis jasa ekspor dan impor yang tersedia\n
        3. 📑 Panduan dokumen ekspor/impor yang diperlukan\n
        4. ☎️ Informasi kontak perusahaan\n
Silakan tanyakan salah satu untuk melanjutkan."""

    elif "perusahaan" in query:
        return "Kami adalah PT Raf Jaya Logistik, penyedia jasa ekspor-impor terpercaya sejak 2020."

    elif "jasa" in query or "layanan" in query:
        return "Kami melayani ekspor-impor barang umum, kargo laut & udara, dan layanan bea cukai."

    elif "kontak" in query:
        return "Hubungi kami via:\n📞 WhatsApp: 0812-3456-7890\n📧 Email: info@rafjayalogistik.co.id"

    elif "dokumen" in query:
        return "Dokumen umum: Invoice, Packing List, Bill of Lading, NPWP, dan tambahan sesuai barang."

    return "Silakan tanyakan tentang perusahaan, jasa, dokumen, atau kontak."

Overwriting bot.py


In [ ]:
%%writefile app.py
import streamlit as st
import time
from bot import customer_service  # fungsi sederhana

st.set_page_config(page_title="RJL Bot", page_icon="🚢")
st.title("🤖 RJL Bot")
st.markdown("> Bot dibuat oleh Ariza Sulthan Rafi")
st.divider()

# Simpan histori chat
if "messages" not in st.session_state:
    st.session_state.messages = []

# Tombol reset chat
if st.button("🔄 Reset Chat"):
    st.session_state.messages = []

# Tampilkan histori sebelumnya
for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        if msg["role"] == "assistant":
            styled = f"""
            <div style="border-left: 4px solid #1976D2; background-color:#eef2f7; padding:10px 12px; border-radius:8px;">
                {msg['content']}
            </div>
            """
            st.markdown(styled, unsafe_allow_html=True)
        else:
            st.markdown(msg["content"], unsafe_allow_html=True)

# Input user
user_input = st.chat_input("Tulis pertanyaan Anda...")

if user_input:
    # Tampilkan dan simpan pesan user
    st.chat_message("user").markdown(user_input)
    st.session_state.messages.append({"role": "user", "content": user_input})

    # Dapatkan jawaban dari fungsi
    bot_reply = customer_service(user_input)

    # Simpan jawaban ke histori
    st.session_state.messages.append({"role": "🤖", "content": bot_reply})

    # Tampilkan jawaban bot dengan efek ngetik
    with st.chat_message("🤖"):
        placeholder = st.empty()
        typed_text = ""

        for char in bot_reply:
            typed_text += char
            placeholder.markdown(f"""
            <div style="border-left: 4px solid #1976D2; background-color:#eef2f7; padding:10px 12px; border-radius:8px;">
                {typed_text}
            </div>
            """, unsafe_allow_html=True)
            time.sleep(0.010)  # kecepatan ketik

Overwriting app.py


In [ ]:
!pip install pyngrok

  Using cached pyngrok-7.2.11-py3-none-any.whl.metadata (9.4 kB)


In [ ]:
from google.colab import userdata
import os

search_token = userdata.get('search_token')
ngrok_token = userdata.get('ngrok_token')
api_token = userdata.get('api_token')

os.environ['SEARCH_TOKEN'] = search_token
os.environ['REPLICATE_API_TOKEN'] = api_token

with open(".env", "w") as f:
    f.write(f"SEARCH_TOKEN={search_token}\n")
    f.write(f"REPLICATE_API_TOKEN={api_token}\n")

In [ ]:
import subprocess
import os
import signal
from pyngrok import ngrok, conf
import time

conf.get_default().auth_token = ngrok_token

# Kill previous ngrok tunnels and streamlit
ngrok.kill()

# Kill any previous streamlit running on 8501
!fuser -k 8501/tcp

# Start streamlit
process = subprocess.Popen(["streamlit", "run", "app.py"])

# Wait for it to spin up
time.sleep(5)

# Start new ngrok tunnel
public_url = ngrok.connect(8501)
print(f"Streamlit app running at: {public_url}")

8501/tcp:             1437
Streamlit app running at: NgrokTunnel: "https://1104-34-148-224-5.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!pip install -qU \
    langchain-core==0.2.29 \
    langgraph==0.2.3 \
    langchain-ollama==0.1.1 \
    semantic-router==0.0.61 \
    pyppeteer==2.0.0 \
    nest-asyncio==1.6.0 \
    praw==7.7.1 \
    cohere==5.5.0 # Add cohere with a specific version
!pip install -U langchain langchain-experimental
!pip install langgraph
!pip install langchain-anthropic
!pip install langchain_community
!pip install replicate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.5/158.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

In [ ]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13281    0 13281    0     0  68285      0 --:--:-- --:--:-- --:--:-- 68107
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!npm install -g pm2
!pm2 start "ollama serve" --name ollama

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼
added 135 packages in 14s
⠼
⠼13 packages are looking for funding
⠼  run `npm fund` for details
⠼
                        -------------

__/\\\\\\\\\\\\\____/\\\\____________/\\\\____/\\\\\\\\\_____
 _\/\\\/////////\\\_\/\\\\\\________/\\\\\\__/\\\///////\\\___
  _\/\\\_______\/\\\_\/\\\//\\\____/\\\//\\\_\///______\//\\\__
   _\/\\\\\\\\\\\\\/__\/\\\\///\\\/\\\/_\/\\\___________/\\\/___
    _\/\\\/////////____\/\\\__\///\\\/___\/\\\________/\\\//_____
     _\/\\\_____________\/\\\____\///_____\/\\\_____/\\\//________
      _\/\\\_____________\/\\\_____________\/\\\___/\\\/___________
       _\/\\\_____________\/\\\_____________\/\\\__/\\\\\\\\\\\\\\\_
        _\///______________\///______________\///__\///////////////__


                          Runtime Edition

        PM2 is a Production Process Manager for Node.js applications
           

In [ ]:
!ollama pull qwen3:4b
!ollama pull qwen3:8b